In [1]:
# importando as primeiras bibliotecas

import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# lendo os dados do arquivo csv pertinente aos vôos

df = pd.read_csv('flights.csv', low_memory=False)

In [ ]:
# checando informações básicas dos tipos de dados presentes em cada coluna

df.info()

Há 31 colunas, 26 com valores numéricos e 5 com textos (strings).

Há 5819079 entradas.

In [3]:
# Selecionar 10% do dataset para facilitar o tempo de processamento das visualizações

df_small = df.sample(frac=0.1, random_state=101)

In [ ]:
df_small.head(5)

In [ ]:
# Checar quantidade de valores únicos

df_small.nunique()

In [4]:
# Retirar os valores não numéricos

df_small_numeric = df_small.drop(['AIRLINE', 'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'CANCELLATION_REASON'],axis=1)

In [ ]:
# Observar estatísticas básicas dos dados numéricos

df_small_numeric.describe().transpose()

In [ ]:
df_small_numeric.isnull().sum()/df_small_numeric.shape[0]*100

In [ ]:
plt.figure(figsize=(12,6))
plt.xticks(rotation=0)
sns.heatmap(df_small_numeric.isnull(), cbar=False, yticklabels=False, cmap='spring')

Com mais de 80% de dados faltantes para as variáveis AIR_SYSTEM_DELAY, SECURITY_DELAY, AIRLINE_DELAY, LATE_AIRCRAFT_DELAY, WEATHER_DELAY, opta-se por retirá-las nesse momento.

In [5]:
df_small_numeric = df_small_numeric.drop(['AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'], axis=1)

A porcentagem de dados faltantes nas outras variáveis é bem pequena, por isso opta-se por retirar os casos nulos restantes.

In [6]:
# Desse modo, não há mais dados faltantes no dataset a ser analisado

df_small_numeric = df_small_numeric.dropna()
# df_small_numeric.isnull().sum()/df_small_numeric.shape[0]*100

In [ ]:
df_small_numeric.head().transpose()

In [ ]:
df_small_numeric.info()

In [7]:
df_small_numeric['DATE'] = pd.to_datetime(df_small_numeric[['YEAR', 'MONTH', 'DAY']])

In [8]:
df_small_numeric = df_small_numeric.drop(['YEAR', 'MONTH', 'DAY'], axis=1)

In [ ]:
df_small_numeric.head().transpose()

In [9]:
df_small_numeric.index = list(range(0,571390))

A partir do tutorial disponível em [Predicting flight delays [Tutorial]](https://www.kaggle.com/fabiendaniel/predicting-flight-delays-tutorial/notebook), utilizam-se funções semelhantes às lá desenvolvidas para formatar a hora prevista de saída (SCHEDULED_TIME).

In [10]:
#_________________________________________________________
# Function that convert the 'HHMM' string to datetime.time
def formatar_hora(dado_orig):
    if dado_orig == 2400: dado_orig = 0
    dado_orig = "{0:04d}".format(int(dado_orig))
    horario = datetime.time(int(dado_orig[0:2]), int(dado_orig[2:4]))
    return horario
#_____________________________________________________________________
# Function that combines a date and time to produce a datetime.datetime
def combinar_data_horario(x):
    if pd.isnull(x[0]) or pd.isnull(x[1]):
        return np.nan
    else:
        return datetime.datetime.combine(x[0],x[1])
#_______________________________________________________________________________
# Function that combine two columns of the dataframe to create a datetime format
def criar_horario_voo(df, col):    
    lista = []
    for index, cols in df[['DATE', col]].iterrows():    
        if pd.isnull(cols[1]):
            lista.append(np.nan)
        elif float(cols[1]) == 2400:
            cols[0] += datetime.timedelta(days=1)
            cols[1] = datetime.time(0,0)
            lista.append(combinar_data_horario(cols))
        else:
            cols[1] = formatar_hora(cols[1])
            lista.append(combinar_data_horario(cols))
    return pd.Series(lista)

In [13]:
df_small_numeric['SCHEDULED_DEPARTURE'] = criar_horario_voo(df_small_numeric, 'SCHEDULED_DEPARTURE')

In [14]:
df_small_numeric['DEPARTURE_TIME'] = df_small_numeric['DEPARTURE_TIME'].apply(formatar_hora)
df_small_numeric['SCHEDULED_ARRIVAL'] = df_small_numeric['SCHEDULED_ARRIVAL'].apply(formatar_hora)
df_small_numeric['ARRIVAL_TIME'] = df_small_numeric['ARRIVAL_TIME'].apply(formatar_hora)

In [15]:
df_small_numeric.to_csv('flights.min.csv')

In [67]:
df_small_numeric.head()

,DAY_OF_WEEK,FLIGHT_NUMBER,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,DATE
0,7,6209,2015-01-25 15:08:00,15:23:00,15.0,25.0,1548.0,121.0,123.0,84.0,588,1712.0,14.0,17:09:00,17:26:00,17.0,0,0,2015-01-25
1,2,3158,2015-01-27 11:30:00,11:23:00,-7.0,31.0,1154.0,93.0,120.0,76.0,438,1310.0,13.0,13:03:00,13:23:00,20.0,0,0,2015-01-27
2,3,505,2015-05-27 10:25:00,10:30:00,5.0,13.0,1043.0,180.0,164.0,145.0,1199,1508.0,6.0,15:25:00,15:14:00,-11.0,0,0,2015-05-27
3,7,5233,2015-05-10 20:00:00,23:52:00,232.0,13.0,5.0,200.0,161.0,144.0,1107,129.0,4.0,22:20:00,01:33:00,193.0,0,0,2015-05-10
4,3,1548,2015-03-04 23:59:00,00:01:00,2.0,15.0,16.0,195.0,180.0,157.0,1452,453.0,8.0,05:14:00,05:01:00,-13.0,0,0,2015-03-04


In [58]:
# plt.figure(figsize=(12,6))
# plt.xticks(rotation=0)
# sns.heatmap(df_small_numeric.isnull(), cbar=False, yticklabels=False, cmap='spring')